# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,botro,7.8525,-5.3106,36.10,7,0,1.81,CI,1738085394
1,1,itoman,26.1247,127.6694,13.21,54,100,8.23,JP,1738085461
2,2,araguaina,-7.1911,-48.2072,32.02,51,43,1.94,BR,1738085463
3,3,yelizovo,53.1873,158.3805,-1.92,50,75,7.00,RU,1738085465
4,4,nacala,-14.5428,40.6728,25.72,89,100,0.31,MZ,1738085466


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Create a map plot for the city_data_df DataFrame, enabling a geogrpahic plot with color based on city names
city_map = city_data_df.hvplot.points(
    "Lng",                     
    "Lat",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500           
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Define ideal weather conditions
temp_condition = (city_data_df["Max Temp"].between(21, 27))  # More efficient than two comparisons
wind_condition = (city_data_df["Wind Speed"] < 4.5)
cloud_condition = (city_data_df["Cloudiness"] == 0)

# Create DataFrame with ideal weather conditions
ideal_cities_df = city_data_df[
    temp_condition &
    wind_condition & 
    cloud_condition
].copy()  # Using .copy() prevents SettingWithCopyWarning

# Drop any rows with null values and display info about remaining cities
ideal_cities_df = ideal_cities_df.dropna()
print(f"Found {len(ideal_cities_df)} cities with ideal weather conditions")
ideal_cities_df

Found 8 cities with ideal weather conditions


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,pisco,-13.7000,-76.2167,25.03,73,0,3.60,PE,1738085521
81,81,callao,-12.0667,-77.1500,24.16,69,0,3.60,PE,1738085563
151,151,okap,19.7578,-72.2042,24.67,82,0,3.96,HT,1738085645
228,228,mahina,-17.5065,-149.4890,26.87,74,0,2.06,PF,1738085740
244,244,duong dong,10.2167,103.9667,25.00,83,0,0.00,VN,1738085759
313,313,aoulef,26.9667,1.0833,22.26,13,0,3.50,DZ,1738085842
420,420,chibuto,-24.6867,33.5306,24.01,75,0,4.38,MZ,1738085980
435,435,timimoun,29.2639,0.2310,24.21,19,0,2.06,DZ,1738085997


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,pisco,PE,-13.7000,-76.2167,73,
81,callao,PE,-12.0667,-77.1500,69,
151,okap,HT,19.7578,-72.2042,82,
228,mahina,PF,-17.5065,-149.4890,74,
244,duong dong,VN,10.2167,103.9667,83,
313,aoulef,DZ,26.9667,1.0833,13,
420,chibuto,MZ,-24.6867,33.5306,75,
435,timimoun,DZ,29.2639,0.2310,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Only need first result
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
total_cities = len(hotel_df)
print(f"Starting hotel search for {total_cities} cities...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    try:
        print(f"Processing city {index+1}/{total_cities}: {row['City']}")
        
        # Get latitude, longitude from the DataFrame
        latitude = row["Lat"]
        longitude = row["Lng"]
        
        # Update parameters for this city
        params["filter"] = f"circle:{longitude},{latitude},{radius}"
        params["bias"] = f"proximity:{longitude},{latitude}"
        
        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"

        # Make API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for HTTP errors
        
        # Process response
        name_address = response.json()
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
        print(f"Found hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Catches errors related to the API request    
    except requests.exceptions.RequestException as e:
        print(f"API request failed for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Catches errors that occur while parsing the API response 
    except (KeyError, IndexError) as e:
        print(f"No hotel found in {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

print("Hotel search completed!")
hotel_df

Starting hotel search for 8 cities...
Processing city 49/8: pisco
Found hotel: La Portada
Processing city 82/8: callao
Found hotel: Casa Ronald
Processing city 152/8: okap
Found hotel: Hispaniola Hotel
Processing city 229/8: mahina
Found hotel: Motu Martin
Processing city 245/8: duong dong
Found hotel: Kim Hông Ngoc
Processing city 314/8: aoulef
Found hotel: دار الضياف
Processing city 421/8: chibuto
Found hotel: DESHENG
Processing city 436/8: timimoun
Found hotel: أناف
Hotel search completed!


,City,Country,Lat,Lng,Humidity,Hotel Name
48,pisco,PE,-13.7000,-76.2167,73,La Portada
81,callao,PE,-12.0667,-77.1500,69,Casa Ronald
151,okap,HT,19.7578,-72.2042,82,Hispaniola Hotel
228,mahina,PF,-17.5065,-149.4890,74,Motu Martin
244,duong dong,VN,10.2167,103.9667,83,Kim Hông Ngoc
313,aoulef,DZ,26.9667,1.0833,13,دار الضياف
420,chibuto,MZ,-24.6867,33.5306,75,DESHENG
435,timimoun,DZ,29.2639,0.2310,19,أناف


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    "Lng",                     
    "Lat",                     
    geo=True,                  
    tiles="CartoLight",        
    size="Humidity",           
    color="City",              
    alpha=0.8,                 
    frame_width=700,           
    frame_height=500, 
    hover_cols=["Hotel Name", "Country"]           
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)